# Serverless

- Name: Aryan Ebrahimpour
- ID: 98726942

In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
#r "nuget: FSharp.Data"
#r "nuget: NodaTime"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

open System
open Plotly.NET
open Plotly.NET.Interactive
open NodaTime
open System.Text.Json
open System.Text.Json.Serialization

Installed Packages FSharp.Data, 4.2.7 MathNet.Numerics, 4.15.0 MathNet.Numerics.FSharp, 4.15.0 NodaTime, 3.0.9 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#load "Serverless.fsx"
open Avestura.Serverless
open MathNet.Numerics.Distributions

In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers

let iterationsCount = 10

let queue = genQueue config_UFD_AW_N 20u
let results = runBatch iterationsCount queue (dynamicContextScheduler)
let qos = qosOf results

In [ ]:
qos |> SimulatorContextBatchQoS.responseTimeChartData |> Chart.Spline

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Visualization of the General.normalize
// function using Weilbul Stretched Function

let x = [ for x in 0. .. 0.1 .. 20 -> (x, General.normalize x) ]

x |> Chart.Line

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Dijestra path finding

let dg = DependencyGraph.init (20u)

(dg.adjMatrix) |> GraphExtensions.printAdjMatrix

let path, distance = dg |> DependencyGraph.dijkestraPath (0u) (5u)
let weights = dg |> DependencyGraph.edgeWeightsFromPath path

path, weights, distance


--------------
|  0 | 909 | 745 | 0 | 637 | 0 | 557 | 0 | 936 | 317 | 209 | 0 | 937 | 0 | 918 | 784 | 0 | 0 | 908 | 453 |
--------------
|  616 | 0 | 787 | 0 | 0 | 781 | 423 | 0 | 0 | 0 | 797 | 0 | 0 | 106 | 738 | 593 | 0 | 49 | 697 | 863 |
--------------
|  344 | 489 | 0 | 198 | 106 | 127 | 0 | 232 | 546 | 0 | 0 | 0 | 960 | 0 | 0 | 220 | 920 | 470 | 0 | 295 |
--------------
|  0 | 0 | 505 | 73 | 0 | 0 | 941 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 722 | 895 | 713 |
--------------
|  426 | 196 | 0 | 763 | 99 | 0 | 0 | 0 | 594 | 601 | 0 | 0 | 0 | 55 | 580 | 0 | 171 | 0 | 0 | 0 |
--------------
|  430 | 0 | 0 | 433 | 994 | 451 | 320 | 767 | 0 | 0 | 0 | 0 | 0 | 596 | 0 | 810 | 325 | 948 | 344 | 819 |
--------------
|  752 | 0 | 0 | 974 | 768 | 809 | 334 | 296 | 0 | 0 | 553 | 955 | 0 | 663 | 237 | 40 | 715 | 54 | 69 | 634 |
--------------
|  492 | 0 | 251 | 0 | 425 | 0 | 0 | 0 | 0 | 0 | 406 | 135 | 681 | 0 | 0 | 60 | 208 | 0 | 825 | 0 |
--------------
|  872 | 831 | 650 | 0 | 985 | 787 | 

In [ ]:
// fsevents
// hiredis
// karma-phantomjs-launcher
// phantomjs-prebuilt
// puppeteer
// serialport
// xml2json
// yeoman-environment
// let dd_impossible = [
//     "fsevents"
//     "hiredis"
//     "karma-phantomjs-launcher"
//     "phantomjs-prebuilt"
//     "puppeteer"
//     "serialport"
//     "xml2json"
//     "yeoman-environment"
// ]
// let dd_1 = PackagesData.dependentUpon.Keys |> List.ofSeq |> List.skipWhile(fun x -> x <> "yeoman-generator")
// PackagesData.downloadAndSaveAllPackageSizes dd_impossible

//dd_1.[1]

In [ ]:
let saturdayData = FrequenciesData.frequencies["saturday"]
Chart.Line(mapKeys saturdayData, mapValues saturdayData)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
open QueueDataGenerator
open QueueFunctionGeneration
open Simulator


let config1 : QueueFunctionGeneration.Options = 
    (IgnoreFrequencyData, IgnoreCoopNetwork, defaultUnawareRandomNormalDepsCount)
let queueData = generateFunctionQueueData config1 6
// let context = runSimulatonWithQueueData "saturday" queueData noMergeScheduler

let x = DependencyGraph(queueData.funcs)


Error: input.fsx (13,15)-(13,32) typecheck error This function takes too many arguments, or is used in a context where a function is not expected

In [ ]:
results |> SimulatorContext.getQoSMeassuresList

index utilization responseTime turnaroundTime cost 0 0.04612887905122879 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 522000000000 0 8 42 0 0 0 5220000000 0.0060416666666666665 0.145 8.7 522 522000 5220000000 522000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 528504263147 0 8 48 504 5042631 504263147 5285042631 0.006116947490127315 0.14680673976305555 8.808404385783334 528.504263147 528504.263147 5285042631.47 528504263147 0.0061562505264666665 1 0.06681392075498076 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 288000000000 0 4 48 0 0 0 2880000000 0.0033333333333333335 0.08 4.8 288 288000 2880000000 288000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 290116136176 0 4 50 116 1161361 116136176 2901161361 0.0033578256501851853 0.08058781560444445 4.835268936266667 290.116136176 290116.136176 2901161361.76 290116136176 0.003454308519888889 2 0.0935589204955372 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 343000000000 0 5 43 0 0 0 3430000000 0.003969907407407407 0.09527777777777778 5.716666666666667 343 343000 3430000000 343000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 347575960282 0 5 47 575 5759602 575960282 3475759602 0.004022869910671296 0.09654887785611112 5.792932671366667 347.575960282 347575.960282 3475759602.82 347575960282 0.004260569386766667 3 0.3125057826459409 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 262000000000 0 4 22 0 0 0 2620000000 0.0030324074074074073 0.07277777777777777 4.366666666666666 262 262000 2620000000 262000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 265031932361 0 4 25 31 319323 31932361 2650319323 0.003067499217141204 0.0736199812113889 4.4171988726833336 265.031932361 265031.932361 2650319323.61 265031932361 0.004283380389422223 4 0.27795652702342755 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 249000000000 0 4 9 0 0 0 2490000000 0.0028819444444444444 0.06916666666666667 4.15 249 249000 2490000000 249000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 251073821109 0 4 11 73 738211 73821109 2510738211 0.002905947003576389 0.06974272808583333 4.18456368515 251.073821109 251073.821109 2510738211.09 251073821109 0.0038636304154000003 5 0.03665024188370652 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 701000000000 0 11 41 0 0 0 70100